In [2]:
using CSV, DataFrames, JuMP, HiGHS
include("CEM.jl")

storageinfo = CSV.read(joinpath("cleaned","storageinfo.csv"), DataFrame)
generatorinfo = CSV.read(joinpath("cleaned","generatorinfo.csv"), DataFrame)
vreinfo = CSV.read(joinpath("cleaned","vreinfo.csv"), DataFrame)
halfhourlydemand = CSV.read(joinpath("cleaned","halfhourlydemand.csv"), DataFrame)
halfhourlyvrecf = CSV.read(joinpath("cleaned","halfhourlyvrecf.csv"), DataFrame)
halfhourlydemand = halfhourlydemand[1:1488,:];

In [4]:
eff_results = Dict()

selected_ldes_ids = storageinfo[
    (storageinfo.name.==" Compressed_air_energy_storage") .| 
    (storageinfo.name.==" Aqueous_sulfur_flow_batteries") .| 
    (storageinfo.name.==" Vanadium_redox_flow_batteries") .|
    (storageinfo.name.==" Multijunction_photovoltaic_thermal_storage"), :id]

4-element Vector{Int64}:
  2
  9
 10
 11

In [ ]:
for s_id in selected_ldes_ids[1:2]
    eff_results[s_id] = []
    for eff in 50:5:95
        eff = eff / 100
        if storageinfo[s_id,:bestCaseEff] < eff
            storageinfo[s_id,:bestCaseEff] = eff
            sol = CEM(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
                budget=20e9, voll=100e3, cd_rho=5, reserve="all", silent=true)
            println(sol)
            push!(eff_results[s_id], (eff, sol.TOC))
        end
    end
    storageinfo = CSV.read(joinpath("cleaned","storageinfo.csv"), DataFrame)
end

In [ ]:
for s_id in selected_ldes_ids[3:4]
    eff_results[s_id] = []
    for eff in 50:5:95
        eff = eff / 100
        if storageinfo[s_id,:bestCaseEff] < eff
            storageinfo[s_id,:bestCaseEff] = eff
            sol = CEM(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
                budget=20e9, voll=100e3, cd_rho=5, reserve="all", silent=true)
            println(sol)
            push!(eff_results[s_id], (eff, sol.TOC))
        end
    end
    storageinfo = CSV.read(joinpath("cleaned","storageinfo.csv"), DataFrame)
end